In [1]:
#第6章/加载tokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('hfl/rbt3')

tokenizer

BertTokenizerFast(name_or_path='hfl/rbt3', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [2]:
#第6章/试编码句子
tokenizer.batch_encode_plus(
    ['明月装饰了你的窗子', '你装饰了别人的梦'],
    truncation=True,
)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [[101, 3209, 3299, 6163, 7652, 749, 872, 4638, 4970, 2094, 102], [101, 872, 6163, 7652, 749, 1166, 782, 4638, 3457, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [3]:
#第6章/从磁盘加载数据集
from datasets import load_from_disk

dataset = load_from_disk('../data/ChnSentiCorp')

#缩小数据规模，便于测试
dataset['train'] = dataset['train'].shuffle().select(range(2000))
dataset['test'] = dataset['test'].shuffle().select(range(100))

dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 100
    })
})

In [4]:
#第6章/编码
# from transformers import AutoTokenizer
#
# tokenizer = AutoTokenizer.from_pretrained('hfl/rbt3')
def f(data):
    return tokenizer.batch_encode_plus(data['text'], truncation=True)


dataset = dataset.map(f,
                      batched=True,
                      batch_size=1000,
                      num_proc=1,
                      remove_columns=['text'])
dataset

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Loading cached processed dataset at D:\workspace\learning-huggingface\data\ChnSentiCorp\validation\cache-4da7acca9537a086.arrow


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1200
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 100
    })
})

In [5]:
#第6章/移除太长的句子
def f(data):
    return [len(i) <= 512 for i in data['input_ids']]


dataset = dataset.filter(f, batched=True, batch_size=1000, num_proc=4)

dataset

Filter (num_proc=4):   0%|          | 0/2000 [00:00<?, ? examples/s]

Loading cached processed dataset at D:\workspace\learning-huggingface\data\ChnSentiCorp\validation\cache-3eeeed8a3981ac50_*_of_00004.arrow


Filter (num_proc=4):   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1975
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1190
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 98
    })
})

In [6]:
#第6章/加载模型
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('hfl/rbt3',
                                                           num_labels=2)

#统计模型参数量
sum([i.nelement() for i in model.parameters()]) / 10000

Some weights of the model checkpoint at hfl/rbt3 were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3

3847.8338

In [7]:
#第6章/模型试算
import torch

#模拟一批数据
data = {
    'input_ids': torch.ones(4, 10, dtype=torch.long),
    'token_type_ids': torch.ones(4, 10, dtype=torch.long),
    'attention_mask': torch.ones(4, 10, dtype=torch.long),
    'labels': torch.ones(4, dtype=torch.long)
}

#模型试算
out = model(**data)

out['loss'], out['logits'].shape

(tensor(0.4832, grad_fn=<NllLossBackward0>), torch.Size([4, 2]))

In [8]:
# #第6章/加载评价指标
# from datasets import load_metric
# metric = load_metric('accuracy')

import evaluate

metric = evaluate.load('accuracy')

In [9]:
#第6章/定义评价函数
import numpy as np
from transformers.trainer_utils import EvalPrediction


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits = logits.argmax(axis=1)
    return {'accuracy': (logits == labels).sum() / len(labels)}
    #return metric.compute(predictions=logits, references=labels)


#模拟输出
eval_pred = EvalPrediction(
    predictions=np.array([[0, 1], [2, 3], [4, 5], [6, 7]]),
    label_ids=np.array([1, 1, 0, 1]),
)

compute_metrics(eval_pred)

{'accuracy': 0.75}

In [11]:
#第6章/定义训练参数
from transformers import TrainingArguments

#定义训练参数
args = TrainingArguments(
    #定义临时数据保存路径
    output_dir='./output_dir',

    #定义测试执行的策略，可取值no、epoch、steps
    evaluation_strategy='steps',

    #定义每隔多少个step执行一次测试
    eval_steps=30,

    #定义模型保存策略，可取值no、epoch、steps
    save_strategy='steps',

    #定义每隔多少个step保存一次
    save_steps=30,

    #定义共训练几个轮次
    num_train_epochs=1,

    #定义学习率
    learning_rate=1e-4,

    #加入参数权重衰减，防止过拟合
    weight_decay=1e-2,

    #定义测试和训练时的批次大小
    per_device_eval_batch_size=16,
    per_device_train_batch_size=16,

    #定义是否要使用gpu训练
    no_cuda=False,
)

In [12]:
#第6章/定义训练器
from transformers import Trainer
from transformers.data.data_collator import DataCollatorWithPadding

#定义训练器
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer),
)

In [13]:
#第6章/测试数据整理函数
data_collator = DataCollatorWithPadding(tokenizer)

#获取一批数据
data = dataset['train'][:5]

#输出这些句子的长度
for i in data['input_ids']:
    print(len(i))

#调用数据整理函数
data = data_collator(data)

#查看整理后的数据
for k, v in data.items():
    print(k, v.shape)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


50
49
85
76
65
input_ids torch.Size([5, 85])
token_type_ids torch.Size([5, 85])
attention_mask torch.Size([5, 85])
labels torch.Size([5])


In [14]:
tokenizer.decode(data['input_ids'][0])

'[CLS] 正 在 刷 新 bios 、 装 xp 系 统 、 装 驱 动 ， 工 程 没 完 ， 不 知 道 长 期 使 用 怎 么 样 。 听 网 上 说 无 线 网 卡 有 些 问 题 ， 有 点 怕 ！ [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [15]:
#第6章/评价模型
trainer.evaluate()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ········


wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\thinkpad/.netrc


{'eval_loss': 0.7289360761642456,
 'eval_accuracy': 0.5204081632653061,
 'eval_runtime': 1.9807,
 'eval_samples_per_second': 49.476,
 'eval_steps_per_second': 3.534}

In [16]:
#第6章/训练
trainer.train()

D:\anaconda3\envs\huggingface\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
30,No log,0.351372,0.836735
60,No log,0.283773,0.897959
90,No log,0.328433,0.877551
120,No log,0.380387,0.836735


TrainOutput(global_step=124, training_loss=0.3940151891400737, metrics={'train_runtime': 6.0259, 'train_samples_per_second': 327.754, 'train_steps_per_second': 20.578, 'total_flos': 69927521442396.0, 'train_loss': 0.3940151891400737, 'epoch': 1.0})

In [17]:
#第6章/从某个存档继续训练
trainer.train(resume_from_checkpoint='./output_dir/checkpoint-90')

Step,Training Loss,Validation Loss,Accuracy
120,No log,0.348675,0.897959


TrainOutput(global_step=124, training_loss=0.0632401704788208, metrics={'train_runtime': 1.4428, 'train_samples_per_second': 1368.867, 'train_steps_per_second': 85.944, 'total_flos': 67822927473180.0, 'train_loss': 0.0632401704788208, 'epoch': 1.0})

In [18]:
#第6章/评价模型
trainer.evaluate()

{'eval_loss': 0.34703460335731506,
 'eval_accuracy': 0.8979591836734694,
 'eval_runtime': 0.0803,
 'eval_samples_per_second': 1219.995,
 'eval_steps_per_second': 87.142,
 'epoch': 1.0}

In [19]:
#第6章/手动保存模型参数
trainer.save_model(output_dir='./output_dir/save_model')

In [20]:
#第6章/手动加载模型参数
import torch

model.load_state_dict(torch.load('./output_dir/save_model/pytorch_model.bin'))

<All keys matched successfully>

In [21]:
#第6章/测试
model.eval()

for i, data in enumerate(trainer.get_eval_dataloader()):
    break

for k, v in data.items():
    data[k] = v.to('cuda')

out = model(**data)
out = out['logits'].argmax(dim=1)

for i in range(16):
    print(tokenizer.decode(data['input_ids'][i], skip_special_tokens=True))
    print('label=', data['labels'][i].item())
    print('predict=', out[i].item())

携 程 定 的 不 带 早 饭 ， 所 以 不 能 评 价 吃 的 。 总 体 来 说 装 修 不 错 ， 没 有 什 么 特 别 大 的 毛 病 。 停 车 先 收 20 快 一 天 ， 最 后 到 前 台 报 销 。 空 调 非 常 足 ， 都 有 点 过 热 了 。 这 个 价 位 住 到 这 样 子 的 酒 店 ， 非 常 好 了 。 感 觉 比 较 适 合 家 庭 出 游 。 上 网 贵 的 就 是 不 要 上 网 了 ， 比 较 黑 。
label= 1
predict= 1
猫 和 老 鼠 的 dvd, 我 在 当 当 网 已 买 过 10 余 次 了 。 除 了 做 为 礼 物 送 给 亲 朋 好 有 的 孩 子 外 ， 还 留 下 自 己 观 看 ， 而 且 是 百 看 不 厌......
label= 0
predict= 0
充 其 量 一 个 度 假 村 而 已 ， 客 房 其 实 并 不 大 ， 不 知 通 过 什 么 手 段 评 上 4 星 了 。 酒 店 近 邻 104 国 道 ， 是 连 套 别 墅 销 售 附 带 的 设 施 。 房 间 装 修 味 道 很 大 ， 其 中 一 个 房 子 竟 然 没 有 窗 户 。 卫 生 间 一 股 臭 味 ， 设 施 还 不 如 3 星 级 酒 店 。 如 果 不 是 去 山 东 路 上 太 晚 ， 怎 么 也 不 会 到 这 个 酒 店 。
label= 0
predict= 0
书 本 的 质 量 很 差 ： 收 到 的 书 本 前 面 几 页 装 订 反 了 ， 有 几 页 本 来 就 没 有 装 订 好 ， 直 接 就 可 以 拿 下 来 。 寄 回 当 当 网 换 货 ， 20 多 天 了 ， 一 点 消 息 都 没 有 ， 写 了 邮 件 通 知 换 货 的 原 因 ， 却 没 有 见 到 任 何 回 复 。 为 了 要 换 货 ， 我 还 要 特 意 去 跑 一 趟 邮 局 。 这 次 买 书 简 直 是 受 气 。 这 样 的 服 务 也 太 差 了 。
label= 0
predict= 0
酒 店 比 较 旧 ， 不 符 合 四 星 标 准 ， 出 行 不 是 很 方 便 。
label= 0
predict= 0
没 有 许 多 网 友 评 价 热 度 高 的 问 题 ， 

In [22]:
#第6章/定义评价函数
import numpy as np
from transformers.trainer_utils import EvalPrediction


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits = logits.argmax(axis=1)
    return {'accuracy': (logits == labels).sum() / len(labels)}
    #return metric.compute(predictions=logits, references=labels)


#模拟输出
eval_pred = EvalPrediction(
    predictions=np.array([[0, 1], [2, 3], [4, 5], [6, 7]]),
    label_ids=np.array([1, 1, 0, 1]),
)

compute_metrics(eval_pred)

{'accuracy': 0.75}

In [23]:
#第6章/定义训练参数
from transformers import TrainingArguments

#定义训练参数
args = TrainingArguments(
    #定义临时数据保存路径
    output_dir='./output_dir',

    #定义测试执行的策略，可取值no、epoch、steps
    evaluation_strategy='steps',

    #定义每隔多少个step执行一次测试
    eval_steps=30,

    #定义模型保存策略，可取值no、epoch、steps
    save_strategy='steps',

    #定义每隔多少个step保存一次
    save_steps=30,

    #定义共训练几个轮次
    num_train_epochs=1,

    #定义学习率
    learning_rate=1e-4,

    #加入参数权重衰减，防止过拟合
    weight_decay=1e-2,

    #定义测试和训练时的批次大小
    per_device_eval_batch_size=16,
    per_device_train_batch_size=16,

    #定义是否要使用gpu训练
    no_cuda=False,
)

In [24]:
#第6章/定义训练器
from transformers import Trainer
from transformers.data.data_collator import DataCollatorWithPadding

#定义训练器
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer),
)

In [25]:
#第6章/测试数据整理函数
data_collator = DataCollatorWithPadding(tokenizer)

#获取一批数据
data = dataset['train'][:5]

#输出这些句子的长度
for i in data['input_ids']:
    print(len(i))

#调用数据整理函数
data = data_collator(data)

#查看整理后的数据
for k, v in data.items():
    print(k, v.shape)

50
49
85
76
65
input_ids torch.Size([5, 85])
token_type_ids torch.Size([5, 85])
attention_mask torch.Size([5, 85])
labels torch.Size([5])


In [26]:
tokenizer.decode(data['input_ids'][0])

'[CLS] 正 在 刷 新 bios 、 装 xp 系 统 、 装 驱 动 ， 工 程 没 完 ， 不 知 道 长 期 使 用 怎 么 样 。 听 网 上 说 无 线 网 卡 有 些 问 题 ， 有 点 怕 ！ [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [27]:
#第6章/评价模型
trainer.evaluate()

{'eval_loss': 0.34703460335731506,
 'eval_accuracy': 0.8979591836734694,
 'eval_runtime': 0.0792,
 'eval_samples_per_second': 1237.951,
 'eval_steps_per_second': 88.425}

In [28]:
#第6章/训练
trainer.train()

D:\anaconda3\envs\huggingface\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
30,No log,0.492852,0.877551
60,No log,0.400477,0.897959
90,No log,0.316759,0.877551
120,No log,0.395314,0.846939


TrainOutput(global_step=124, training_loss=0.24466546889274352, metrics={'train_runtime': 5.4675, 'train_samples_per_second': 361.227, 'train_steps_per_second': 22.68, 'total_flos': 69927521442396.0, 'train_loss': 0.24466546889274352, 'epoch': 1.0})

In [29]:
#第6章/从某个存档继续训练
trainer.train(resume_from_checkpoint='./output_dir/checkpoint-90')

Step,Training Loss,Validation Loss,Accuracy
120,No log,0.244510,0.928571


TrainOutput(global_step=124, training_loss=0.04930720790739982, metrics={'train_runtime': 1.4296, 'train_samples_per_second': 1381.513, 'train_steps_per_second': 86.738, 'total_flos': 67822927473180.0, 'train_loss': 0.04930720790739982, 'epoch': 1.0})

In [30]:
#第6章/评价模型
trainer.evaluate()

{'eval_loss': 0.24330143630504608,
 'eval_accuracy': 0.9285714285714286,
 'eval_runtime': 0.0824,
 'eval_samples_per_second': 1189.336,
 'eval_steps_per_second': 84.953,
 'epoch': 1.0}

In [31]:
#第6章/手动保存模型参数
trainer.save_model(output_dir='./output_dir/save_model')

In [32]:
#第6章/手动加载模型参数
import torch

model.load_state_dict(torch.load('./output_dir/save_model/pytorch_model.bin'))

<All keys matched successfully>

In [33]:
#第6章/测试
model.eval()

for i, data in enumerate(trainer.get_eval_dataloader()):
    break

for k, v in data.items():
    data[k] = v.to('cuda')

out = model(**data)
out = out['logits'].argmax(dim=1)

for i in range(16):
    print(tokenizer.decode(data['input_ids'][i], skip_special_tokens=True))
    print('label=', data['labels'][i].item())
    print('predict=', out[i].item())

携 程 定 的 不 带 早 饭 ， 所 以 不 能 评 价 吃 的 。 总 体 来 说 装 修 不 错 ， 没 有 什 么 特 别 大 的 毛 病 。 停 车 先 收 20 快 一 天 ， 最 后 到 前 台 报 销 。 空 调 非 常 足 ， 都 有 点 过 热 了 。 这 个 价 位 住 到 这 样 子 的 酒 店 ， 非 常 好 了 。 感 觉 比 较 适 合 家 庭 出 游 。 上 网 贵 的 就 是 不 要 上 网 了 ， 比 较 黑 。
label= 1
predict= 1
猫 和 老 鼠 的 dvd, 我 在 当 当 网 已 买 过 10 余 次 了 。 除 了 做 为 礼 物 送 给 亲 朋 好 有 的 孩 子 外 ， 还 留 下 自 己 观 看 ， 而 且 是 百 看 不 厌......
label= 0
predict= 1
充 其 量 一 个 度 假 村 而 已 ， 客 房 其 实 并 不 大 ， 不 知 通 过 什 么 手 段 评 上 4 星 了 。 酒 店 近 邻 104 国 道 ， 是 连 套 别 墅 销 售 附 带 的 设 施 。 房 间 装 修 味 道 很 大 ， 其 中 一 个 房 子 竟 然 没 有 窗 户 。 卫 生 间 一 股 臭 味 ， 设 施 还 不 如 3 星 级 酒 店 。 如 果 不 是 去 山 东 路 上 太 晚 ， 怎 么 也 不 会 到 这 个 酒 店 。
label= 0
predict= 0
书 本 的 质 量 很 差 ： 收 到 的 书 本 前 面 几 页 装 订 反 了 ， 有 几 页 本 来 就 没 有 装 订 好 ， 直 接 就 可 以 拿 下 来 。 寄 回 当 当 网 换 货 ， 20 多 天 了 ， 一 点 消 息 都 没 有 ， 写 了 邮 件 通 知 换 货 的 原 因 ， 却 没 有 见 到 任 何 回 复 。 为 了 要 换 货 ， 我 还 要 特 意 去 跑 一 趟 邮 局 。 这 次 买 书 简 直 是 受 气 。 这 样 的 服 务 也 太 差 了 。
label= 0
predict= 0
酒 店 比 较 旧 ， 不 符 合 四 星 标 准 ， 出 行 不 是 很 方 便 。
label= 0
predict= 0
没 有 许 多 网 友 评 价 热 度 高 的 问 题 ， 